# Electricity Price Forecasting on the German day-ahead market

This notebook is the main interface to the associated EPF library. Various parameters can be adjusted within the library via the Config file. A more detailed description of the individual parameters can be found within the configuration classes. In the “exploratory_analysis” notebook, the raw data sets are analyzed exploratively and the results are displayed visually. Based on these results, features from the data sets were specifically selected for further use in the deep learning pipeline.

The forecasting pipeline is built to automatically perform data preprocessing including data cleaning, outlier removal and seasonal decomposition. Within the configuration feature engineering can be toggled on and of for each feature. Forecasting is done with two different models, that can be retrained and saved anytime. The models available are an LSTM and GRU. They all perform multi-step ahead, single shot forecasts. This means by default each forecast contains 24 time steps into the future, that are forecasted together in one computation.

In [ ]:
from epf.pipeline import EpfPipeline

pipeline = EpfPipeline()

In [ ]:
pipeline.train('test', overwrite=True, prep_data=False, use_tuned_hyperparams=False)

In [ ]:
pipeline.evaluate('lstm_all_features_hl0_drY')

In [ ]:
import pickle as pkl

with open("../data/processed/performance.pkl", "rb") as f:
    performance = pkl.load(f)

with open("../data/processed/val_performance.pkl", "rb") as f:
    val_performance = pkl.load(f)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

x = np.arange(len(performance))
width = 0.3

metric_name = 'mean_absolute_error'
val_mae = [v[metric_name] for v in val_performance.values()]
test_mae = [v[metric_name] for v in performance.values()]

plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs) normed')
_ = plt.legend()

In [ ]:
import pickle as pkl

with open("../models/lstm_all_features.pkl", "rb") as f:
    model_obj = pkl.load(f)

window = model_obj['window']
model = model_obj['best_model']
window.plot(model)

In [ ]:
from epf.config import PREDICTIONS_DIR, MODELS_DIR

model_path = MODELS_DIR / "lstm_all_features.keras"
predictions_dir = PREDICTIONS_DIR

In [ ]:
pipeline.predict(data=window.test, model_path=model_path, predictions_dir=predictions_dir)